In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader


In [2]:
# Load the CSV file
data = pd.read_csv('Diabetes-Classification.csv')

In [4]:
data = data.drop("PatientID", axis=1)

In [6]:
# Split into features and labels
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

In [9]:
# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [10]:
# Create a custom dataset
#Create a custom PyTorch dataset to handle the data.
class CustomDataset(Dataset):
    def __init__(self, features, labels):
        self.features = torch.tensor(features, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.long)

    def __len__(self):
        return len(self.features)

    def __getitem__(self, index):
        return self.features[index], self.labels[index]

# Create training and testing datasets
train_dataset = CustomDataset(X_train, y_train)
test_dataset = CustomDataset(X_test, y_test)


In [11]:
#Define the model
#Define your neural network model by inheriting from nn.Module and 
#implementing the forward method.

class Classifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(Classifier, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

# Define model parameters
input_size = X.shape[1]  # Number of features
hidden_size = 64  # Number of neurons in the hidden layer
num_classes = 2  # Number of output classes

# Create the model
model = Classifier(input_size, hidden_size, num_classes)


In [12]:
# Define the loss function and optimizer
#Choose an appropriate loss function and optimizer for your model.

loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [13]:
#Train the model
# Define the training parameters
batch_size = 64
num_epochs = 10

# Create data loaders for batching
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Training loop
for epoch in range(num_epochs):
    for batch_features, batch_labels in train_loader:
        # Forward pass
        outputs = model(batch_features)
        loss = loss_function(outputs, batch_labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Print the loss after every epoch
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}")

print("Training finished!")


Epoch [1/10], Loss: 0.5856990218162537
Epoch [2/10], Loss: 0.5871492028236389
Epoch [3/10], Loss: 0.5399693846702576
Epoch [4/10], Loss: 0.40129056572914124
Epoch [5/10], Loss: 0.5395045280456543
Epoch [6/10], Loss: 0.4372243583202362
Epoch [7/10], Loss: 0.435557097196579
Epoch [8/10], Loss: 0.4815484583377838
Epoch [9/10], Loss: 0.516132652759552
Epoch [10/10], Loss: 0.6835353374481201
Training finished!


In [19]:
# Create a data loader for testing
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Evaluation loop
model.eval()  # Set the model to evaluation mode
correct = 0
total = 0
predictions = []

with torch.no_grad():
    for batch_features, batch_labels in test_loader:
        # Forward pass
        outputs = model(batch_features)

        # Convert outputs to probabilities using softmax
        probabilities = torch.softmax(outputs, dim=1)

        # Get predicted labels
        _, predicted_labels = torch.max(probabilities, 1)

        # Update total and correct predictions
        total += batch_labels.size(0)
        correct += (predicted_labels == batch_labels).sum().item()

        # Collect predictions
        predictions.extend(predicted_labels.tolist())

# Calculate accuracy
accuracy = correct / total
print(f"Test Accuracy: {accuracy * 100}%")

# Make predictions on new data
new_data = [[3, 100, 78 ,25], [304, 42.60, 0.07,30]]  # New data to predict
new_data_tensor = torch.tensor(new_data, dtype=torch.float32)
model.eval()  # Set the model to evaluation mode

with torch.no_grad():
    outputs = model(new_data_tensor)
    probabilities = torch.softmax(outputs, dim=1)
    _, predicted_labels = torch.max(probabilities, 1)

print("Predicted labels for new data:", predicted_labels.tolist())


Test Accuracy: 77.23333333333333%


RuntimeError: mat1 and mat2 shapes cannot be multiplied (2x4 and 8x64)